In [1]:
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import eq

from eq.catalogs.Combined_Catalog import CombinedCatalog

from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [2]:
catalogs = []
anss = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=100,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    include_depth=False
)
catalogs.append(anss)
scedc = eq.catalogs.SCEDC(include_depth=False)
catalogs.append(scedc)
white = eq.catalogs.White()
catalogs.append(white)
# qtm = eq.catalogs.QTMSaltonSea()
# catalogs.append(qtm)


Loading existing catalog from /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog.
Loading existing catalog from /home/gcl/RA/jonahk/recast/data/SCEDC.
Loading existing catalog from /home/gcl/RA/jonahk/recast/data/White.


/home/gcl/RA/jonahk/recast/eq/data/sequence.py:213: UserWarning: Found 1 zero inter-event times in the sequence. This violates fundamental assumptions of TPP models and may lead to incorrect log-likelihood values.
  warnings.warn(


In [3]:
combined_catalog = CombinedCatalog(catalogs, [.7, .15, .15])

Loading existing catalog from /home/gcl/RA/jonahk/recast/notebooks/Jonah/combined.
207
145.0
64
31.0
31
31.0
31


/home/gcl/RA/jonahk/recast/eq/data/sequence.py:213: UserWarning: Found 2 zero inter-event times in the sequence. This violates fundamental assumptions of TPP models and may lead to incorrect log-likelihood values.
  warnings.warn(


In [4]:
combined_catalog.combined_info

{'ANSS_MultiCatalog_test': InMemoryDataset(20),
 'ANSS_MultiCatalog_train': InMemoryDataset(60),
 'ANSS_MultiCatalog_val': InMemoryDataset(20),
 'SCEDC_test': InMemoryDataset(1),
 'SCEDC_train': InMemoryDataset(1),
 'SCEDC_val': InMemoryDataset(1),
 'White_test': InMemoryDataset(1),
 'White_train': InMemoryDataset(1),
 'White_val': InMemoryDataset(1),
 'combined_train': InMemoryDataset(126),
 'combined_val': InMemoryDataset(95),
 'combined_test': InMemoryDataset(104)}

In [4]:
#combined training and val data
combined_train = combined_catalog.combined_info['combined_train'].get_dataloader( batch_size=50, shuffle=False, num_workers=64)
combined_val = combined_catalog.combined_info['combined_val'].get_dataloader( batch_size=50, shuffle=False, num_workers=64)

#single training data and val data
scedc_train = combined_catalog.combined_info['SCEDC_train'].get_dataloader( batch_size=1, shuffle=False)
# white_train = combined_catalog.combined_info['White_train'].get_dataloader( batch_size=1, shuffle=True)
anss_train = combined_catalog.combined_info['ANSS_MultiCatalog_train'].get_dataloader( batch_size=5, shuffle=False)
scedc_val = combined_catalog.combined_info['SCEDC_val'].get_dataloader( batch_size=1, shuffle=False)
# white_val= combined_catalog.combined_info['White_val'].get_dataloader( batch_size=1, shuffle=True)
anss_val= combined_catalog.combined_info['ANSS_MultiCatalog_val'].get_dataloader( batch_size=5, shuffle=False)

# single catalog test data
scedc_test = combined_catalog.combined_info['SCEDC_test'].get_dataloader( batch_size=1, shuffle=False)
# white_test = combined_catalog.combined_info['White_test'].get_dataloader( batch_size=1, shuffle=True)
anss_test = combined_catalog.combined_info['ANSS_MultiCatalog_test'].get_dataloader( batch_size=5, shuffle=False)

In [5]:
T = combined_catalog.combined_info['combined_train'].sequences[0].t_end
N = np.mean([len(seq) for seq in combined_catalog.combined_info['combined_train'].sequences])
mag_mean = np.mean([combined_catalog.combined_info['combined_train'].sequences[0].mag.mean().item() for seq in combined_catalog.combined_info['combined_train']])
tau_mean = T/N
mag_completness = 0.6

combined_model = eq.models.RecurrentTPP(
    tau_mean = tau_mean,
    learning_rate=1e-3,
)

In [6]:
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping, progress_bar],log_every_n_steps=1, 
accumulate_grad_batches=20)

trainer.fit(combined_model, combined_train, combined_val)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.5 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

In [ ]:
checkpoint.best_model_path
combined_model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(combined_model, anss_test)

NameError: name 'checkpoint' is not defined

In [9]:
T = combined_catalog.combined_info['SCEDC_train'][0].t_end

N = np.mean([len(seq) for seq in combined_catalog.combined_info['SCEDC_train']])
mag_mean = np.mean([combined_catalog.combined_info['SCEDC_train'][0].mag.mean().item() for seq in combined_catalog.combined_info['SCEDC_train']])
tau_mean = T/N

single_model = eq.models.RecurrentTPP(
    tau_mean = tau_mean,
    learning_rate=1e-3,
)

    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping, progress_bar],log_every_n_steps=1)

trainer.fit(single_model, scedc_train, scedc_val)

trainer.test(single_model, scedc_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.5 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=1000` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    -14.237809181213379    │
│      total_test_loss      │    -14.237809181213379    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': -14.237809181213379, 'total_test_loss': -14.237809181213379}]